In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import re
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix 

In [3]:
books = pd.read_csv("G:\Document_G\CE_DA\Books.csv")
books.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_14888\173185268.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("G:\Document_G\CE_DA\Books.csv")


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
rating = pd.read_csv("G:\Document_G\CE_DA\Ratings.csv")
rating.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
users = pd.read_csv(r"G:\Document_G\CE_DA\Users.csv")
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [6]:
print(f"Books Shape: {books.shape}\nRatings Shape: {rating.shape}\nUsers Shape: {users.shape}")

Books Shape: (271360, 8)
Ratings Shape: (1149780, 3)
Users Shape: (278858, 3)


In [7]:
print(f"Null value in Books :\n{books.isnull().sum()}\nNull value in Ratings :\n{rating.isnull().sum()}\nNull value in Users :\n{users.isnull().sum()}")

Null value in Books :
ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64
Null value in Ratings :
User-ID        0
ISBN           0
Book-Rating    0
dtype: int64
Null value in Users :
User-ID          0
Location         0
Age         110762
dtype: int64


In [8]:
books_data = books.merge(rating, on = "ISBN")
books_data.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0


In [9]:
df = books_data.copy()
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.drop(columns=["ISBN", "Year-Of-Publication", "Publisher", "Image-URL-S", "Image-URL-M"], axis=1, inplace=True)
df.drop(df[df["Book-Rating"] == 0].index, inplace=True)
df["Book-Title"] = df["Book-Title"].apply(lambda x: re.sub("[\W_]+", " ", x).strip())
df.head()

,Book-Title,Book-Author,Image-URL-L,User-ID,Book-Rating
1,Clara Callan,Richard Bruce Wright,http://images.amazon.com/images/P/0002005018.0...,8,5
3,Clara Callan,Richard Bruce Wright,http://images.amazon.com/images/P/0002005018.0...,11676,8
5,Clara Callan,Richard Bruce Wright,http://images.amazon.com/images/P/0002005018.0...,67544,8
8,Clara Callan,Richard Bruce Wright,http://images.amazon.com/images/P/0002005018.0...,116866,9
9,Clara Callan,Richard Bruce Wright,http://images.amazon.com/images/P/0002005018.0...,123629,9


# Popular Based

In [10]:
def popular_based(df, n):
    rating_count = df.groupby(["Book-Title"])[["Book-Rating"]].count().reset_index()
    rating_count.rename(columns={"Book-Rating":"NumberofVotes"}, inplace=True)
    
    rating_mean = df.groupby(["Book-Title"])[["Book-Rating"]].mean().reset_index()
    rating_mean.rename(columns={"Book-Rating":"Average_ratings"}, inplace=True)

    popular_books = rating_count.merge(rating_mean, on = "Book-Title")
    c = popular_books["Average_ratings"].mean()
    m = popular_books["NumberofVotes"].quantile(0.9)
    
    def weighted_rating(x):
        v = x['NumberofVotes']
        R = x['Average_ratings']
        return ((v * R) + (m * c)) / (v + m)

    popular_books = popular_books[popular_books["NumberofVotes"] >= 250]
    popular_books["Popularity"] = popular_books.apply(weighted_rating, axis=1)
    popular_books = popular_books.sort_values(by="Popularity", ascending=False).reset_index(drop=True).head(n)

    return popular_books.round(1)

In [11]:
# top_rate = list(recommend_books["Book-Title"])
# top_five = []
# for i in range(len(top_rate)):
#     top_five.append(round(df[df["Book-Title"] == top_rate[i]]["Book-Rating"].mean(), 1))
# recommend_books["Rating"] = pd.DataFrame(top_five)
# recommend_books

In [12]:
popular_recommend = popular_based(df, 10)
popular_recommend

,Book-Title,NumberofVotes,Average_ratings,Popularity
0,Harry Potter and the Prisoner of Azkaban Book 3,277,9.0,9.0
1,To Kill a Mockingbird,267,9.0,9.0
2,Harry Potter and the Sorcerer s Stone Harry Po...,315,8.9,8.9
3,Harry Potter and the Chamber of Secrets Book 2,326,8.8,8.8
4,Tuesdays with Morrie An Old Man a Young Man an...,250,8.6,8.6
5,The Secret Life of Bees,406,8.5,8.5
6,The Da Vinci Code,494,8.4,8.4
7,The Lovely Bones A Novel,707,8.2,8.2
8,The Red Tent Bestselling Backlist,383,8.2,8.2
9,Where the Heart Is Oprah s Book Club Paperback,295,8.1,8.1


# Item Based Collaborative Filtering #

In [13]:
title = "The Da Vinci Code"

In [121]:
def item_based(title):
    
    if title in df["Book-Title"].values:
        book = pd.DataFrame(df["Book-Title"].value_counts())
        most_review = book[book["Book-Title"] >= 200].index
        most_review = df[df["Book-Title"].isin(most_review)]

        if title not in most_review["Book-Title"].values:
            try_this = pd.Series(most_review["Book-Title"].unique()).sample(3).values
            print(f"No recommendation for this book.\n\nMay be you choose try this!\n")
            print(f"> {try_this[0]}\n> {try_this[1]}\n> {try_this[2]}")
            return
        else:
            most_review_item = most_review.copy()
            most_review_item = most_review_item.pivot_table(values="Book-Rating", index="User-ID", columns="Book-Title")

            recommend_books = pd.DataFrame(most_review_item.corrwith(most_review_item[title]).sort_values(ascending=False)).reset_index(drop=False)
            recommend_books.rename(columns={0:"Similarity"}, inplace=True)
            
            if title in list(recommend_books["Book-Title"]):
                recommend_books = recommend_books.drop(recommend_books[recommend_books["Book-Title"] == title].index)

            recommend_books = recommend_books[recommend_books["Similarity"] > 0.3] # Similarity threshold = 0.3
            temp = pd.DataFrame(df[df["Book-Title"].isin(list(recommend_books["Book-Title"]))].groupby("Book-Title")["Book-Rating"].mean()).reset_index()
            recommend_books = recommend_books.merge(temp, on="Book-Title")
            recommend_books = recommend_books[recommend_books["Book-Rating"] > 5]
            recommend_books = recommend_books.head(10).round({"Book-Rating": 1}).reset_index(drop=True)
    
    else:
        return "We could not find your book :("
    
    return recommend_books

In [122]:
item_based_recommend = item_based(title)
item_based_recommend

,Book-Title,Similarity,Book-Rating
0,Where the Heart Is Oprah s Book Club Paperback,0.834638,8.1
1,Fahrenheit 451,0.768273,8.6
2,1st to Die A Novel,0.644199,7.7
3,Harry Potter and the Order of the Phoenix Book 5,0.611644,9.0
4,Summer Sisters,0.564726,7.5
5,Snow Falling on Cedars,0.535745,7.8
6,The Five People You Meet in Heaven,0.484228,8.0
7,She s Come Undone Oprah s Book Club,0.452418,7.7
8,The Red Tent Bestselling Backlist,0.393759,8.2
9,Divine Secrets of the Ya Ya Sisterhood A Novel,0.379744,7.8


# User Based Collaborative Filtering #

In [10]:
book = df.groupby("Book-Title").agg(Average_Rating = ("Book-Rating", "mean"), Rating_count = ("Book-Rating", "count")).reset_index()
book = book[book["Rating_count"] >= 200].sort_values(by="Rating_count", ascending=False)
book.head(15)

,Book-Title,Average_Rating,Rating_count
109465,The Lovely Bones A Novel,8.185290,707
131333,Wild Animus,4.390706,581
101988,The Da Vinci Code,8.439271,494
115394,The Secret Life of Bees,8.477833,406
111177,The Nanny Diaries A Novel,7.437659,393
114167,The Red Tent Bestselling Backlist,8.182768,383
29695,Divine Secrets of the Ya Ya Sisterhood A Novel,7.835079,382
15581,Bridget Jones s Diary,7.625995,377
2993,A Painted House,7.398907,366
60282,Life of Pi,8.080357,336


In [11]:
top_review = pd.DataFrame.merge(df, book, on="Book-Title", how="inner")
top_review.head()

,Book-Title,Book-Author,Image-URL-L,User-ID,Book-Rating,Average_Rating,Rating_count
0,The Testament,John Grisham,http://images.amazon.com/images/P/0440234743.0...,3329,8,7.517241,261
1,The Testament,John Grisham,http://images.amazon.com/images/P/0440234743.0...,7346,9,7.517241,261
2,The Testament,John Grisham,http://images.amazon.com/images/P/0440234743.0...,7352,8,7.517241,261
3,The Testament,John Grisham,http://images.amazon.com/images/P/0440234743.0...,9419,5,7.517241,261
4,The Testament,John Grisham,http://images.amazon.com/images/P/0440234743.0...,11224,6,7.517241,261


In [11]:
book_matrix = top_review.pivot_table(values="Book-Rating", index="User-ID", columns="Book-Title")
book_matrix

Book-Title,1st to Die A Novel,A Painted House,A Time to Kill,Angels amp Demons,Bridget Jones s Diary,Divine Secrets of the Ya Ya Sisterhood A Novel,Dreamcatcher,Fahrenheit 451,Girl with a Pearl Earring,Good in Bed,...,The Red Tent Bestselling Backlist,The Secret Life of Bees,The Street Lawyer,The Summons,The Testament,Timeline,To Kill a Mockingbird,Tuesdays with Morrie An Old Man a Young Man and Life s Greatest Lesson,Where the Heart Is Oprah s Book Club Paperback,Wild Animus
User-ID,,,,,,,,,,,,,,,,,,,,,
26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278554,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278633,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
from surprise import accuracy, Dataset, SVD, Reader
from surprise import model_selection, KNNBasic

In [33]:
reader = Reader(rating_scale=(0, 10)) # Rating scale from 0 to 10
data = Dataset.load_from_df(top_review[["User-ID", "Book-Title", "Book-Rating"]], reader)
trainset, testset = model_selection.train_test_split(data, test_size=0.25, random_state=42)

In [34]:
algo = SVD()
algo.fit(trainset)
predict = algo.test(testset)
print(accuracy.rmse(predict))
print(accuracy.mae(predict))

RMSE: 1.6871
1.6870893971547354
MAE:  1.3130
1.3129829216669595


In [35]:
predict_user = pd.DataFrame(predict, columns=["uid", "iid", "rui", "est", "detail"])
temp = predict_user.groupby(["uid","iid"])[["rui"]].mean().reset_index()
predict_user.drop_duplicates(subset=["uid","iid"], inplace=True)
predict_user = predict_user.drop(columns="rui", axis=1).merge(temp, on=["uid", "iid"])

test_user = pd.DataFrame(testset, columns=["User-ID", "Book-Title", "Book-Rating"])

In [36]:
def top_k_precision(predict, actual):
    relevant = len(set(predict["iid"]) & set(actual["Book-Title"]))
    recommend = len(list(predict["iid"]))
    top_k = relevant / recommend
    return top_k

In [40]:
topK = []
k = 10
for user in list(predict_user["uid"]):
    prediction = predict_user[predict_user["uid"] == user].sort_values(by="est", ascending=False).head(k)
    actual_read = test_user[test_user["User-ID"] == user].sort_values(by="Book-Rating", ascending=False).head(k)
    user_top_k = top_k_precision(prediction, actual_read)
    topK.append(user_top_k)
print(f"Top-k precision for this model is: {np.mean(topK)}")

Top-k precision for this model is: 0.996957123098202


In [162]:
# User based collaborative with surprise library
"""
sim_options = {
    "name": "cosine",
    "user_based": True,  
}
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)
predict = algo.test(testset)
accuracy.rmse(predict)
accuracy.mae(predict)
"""

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.8406
MAE:  1.3968


1.3968387860290412

# User Based Collaborative Filtering (Manual without surprise library)

In [43]:
book_norm = book_matrix.subtract(book_matrix.mean(axis=1), axis="rows")
book_norm

Book-Title,1st to Die A Novel,A Painted House,A Time to Kill,Angels amp Demons,Bridget Jones s Diary,Divine Secrets of the Ya Ya Sisterhood A Novel,Dreamcatcher,Fahrenheit 451,Girl with a Pearl Earring,Good in Bed,...,The Red Tent Bestselling Backlist,The Secret Life of Bees,The Street Lawyer,The Summons,The Testament,Timeline,To Kill a Mockingbird,Tuesdays with Morrie An Old Man a Young Man and Life s Greatest Lesson,Where the Heart Is Oprah s Book Club Paperback,Wild Animus
User-ID,,,,,,,,,,,,,,,,,,,,,
26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278554,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278633,NaN,NaN,NaN,NaN,-5.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
similarity_matrix = cosine_similarity(book_norm.fillna(0))
similarity_matrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.35007002],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.35007002, 0.        ,
        1.        ]])

In [45]:
similarity_users = pd.DataFrame(similarity_matrix, index=book_matrix.index, columns=book_matrix.index)
similarity_users.head()

User-ID,26,51,91,114,183,242,243,244,254,256,...,278469,278535,278541,278543,278550,278552,278554,278633,278698,278843
User-ID,,,,,,,,,,,,,,,,,,,,,
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
user_id = 31556
similarity_users.drop(index=user_id, inplace=True)
similarity_users.head()

User-ID,26,51,91,114,183,242,243,244,254,256,...,278469,278535,278541,278543,278550,278552,278554,278633,278698,278843
User-ID,,,,,,,,,,,,,,,,,,,,,
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
min_threshold = 0.3
n = 10
user_id = 31556
similar_users = similarity_users[similarity_users[user_id] > min_threshold][user_id].sort_values(ascending=False).head(n)
if similar_users.empty:
    print("This user have no similar users")
similar_users

User-ID
201526    0.886076
240643    0.886076
177432    0.814092
211919    0.789365
66942     0.767496
257700    0.753115
208815    0.715834
37400     0.711057
14456     0.666973
184904    0.654921
Name: 31556, dtype: float64

In [38]:
user_id_read = book_matrix[book_matrix.index == user_id].dropna(axis=1, how="all")
user_id_read

Book-Title,Girl with a Pearl Earring,Harry Potter and the Sorcerer s Stone Harry Potter Paperback,The Catcher in the Rye,To Kill a Mockingbird
User-ID,,,,
31556,8.0,8.0,5.0,10.0


In [39]:
similar_users_read = book_matrix[book_matrix.index.isin(similar_users.index)].dropna(axis=1, how="all")
similar_users_read

Book-Title,A Painted House,Fahrenheit 451,Good in Bed,Harry Potter and the Chamber of Secrets Book 2,Harry Potter and the Goblet of Fire Book 4,Harry Potter and the Order of the Phoenix Book 5,Harry Potter and the Prisoner of Azkaban Book 3,Harry Potter and the Sorcerer s Stone Harry Potter Paperback,Interview with the Vampire,Jurassic Park,...,The Catcher in the Rye,The Fellowship of the Ring The Lord of the Rings Part 1,The Five People You Meet in Heaven,The Joy Luck Club,The Lovely Bones A Novel,The Notebook,The Pilot s Wife A Novel,The Secret Life of Bees,Timeline,To Kill a Mockingbird
User-ID,,,,,,,,,,,,,,,,,,,,,
14456,NaN,NaN,NaN,10.0,10.0,NaN,10.0,NaN,NaN,NaN,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
66942,NaN,NaN,NaN,9.0,9.0,7.0,9.0,9.0,NaN,9.0,...,5.0,NaN,NaN,10.0,8.0,NaN,NaN,7.0,NaN,10.0
177432,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,10.0
184904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,9.0,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201526,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
208815,NaN,NaN,NaN,10.0,10.0,10.0,10.0,10.0,NaN,NaN,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
211919,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,NaN,NaN,NaN,NaN,7.0,8.0,NaN,NaN,10.0
240643,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0


In [40]:
similar_users_read.drop(columns=user_id_read.columns, axis=1, inplace=True, errors="ignore")
similar_users_read

Book-Title,A Painted House,Fahrenheit 451,Good in Bed,Harry Potter and the Chamber of Secrets Book 2,Harry Potter and the Goblet of Fire Book 4,Harry Potter and the Order of the Phoenix Book 5,Harry Potter and the Prisoner of Azkaban Book 3,Interview with the Vampire,Jurassic Park,Life of Pi,She s Come Undone Oprah s Book Club,Snow Falling on Cedars,The Fellowship of the Ring The Lord of the Rings Part 1,The Five People You Meet in Heaven,The Joy Luck Club,The Lovely Bones A Novel,The Notebook,The Pilot s Wife A Novel,The Secret Life of Bees,Timeline
User-ID,,,,,,,,,,,,,,,,,,,,
14456,NaN,NaN,NaN,10.0,10.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66942,NaN,NaN,NaN,9.0,9.0,7.0,9.0,NaN,9.0,8.0,8.0,NaN,NaN,NaN,10.0,8.0,NaN,NaN,7.0,NaN
177432,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0
184904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201526,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208815,NaN,NaN,NaN,10.0,10.0,10.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
211919,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,8.0,NaN,NaN
240643,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
book_pred_score = {}
user_id_mean = user_id_read.T.mean().values[0]
book_similarity = dict(similar_users)
book_mean = dict(similar_users_read.T.mean())

for i in similar_users_read.columns:
    book_rating = similar_users_read[i]
    sim_de = 0
    total = 0
    for j in book_rating.index:
        if pd.isna(book_rating[j]) == False:
            sim_nu = book_similarity[j] * (book_rating[j] - book_mean[j])
            total += sim_nu
            sim_de += book_similarity[j]
    
    book_pred_score[i] = user_id_mean + (total / sim_de)

book_pred_score = pd.DataFrame(book_pred_score.items(), columns=["Book-Title", "Predict Score"]).sort_values(by="Predict Score", ascending=False)
book_pred_score.round(1).reset_index(drop=True).head(10)

,Book-Title,Predict Score
0,The Joy Luck Club,9.4
1,Good in Bed,8.8
2,The Fellowship of the Ring The Lord of the Rin...,8.4
3,Jurassic Park,8.4
4,Fahrenheit 451,8.1
5,Harry Potter and the Chamber of Secrets Book 2,8.0
6,Harry Potter and the Goblet of Fire Book 4,8.0
7,Harry Potter and the Prisoner of Azkaban Book 3,8.0
8,A Painted House,7.8
9,The Pilot s Wife A Novel,7.8


In [ ]:
# book_score = {}
# for i in similar_users_read.columns:
#     book_rating = similar_users_read[i]
#     count = 0
#     total = 0
#     for j in book_rating.index:
#         if pd.isna(book_rating[j]) == False:
#             score = similar_users[j] * book_rating[j]
#             total += score
#             count += 1
    
#     book_score[i] = total / count

# book_score = pd.DataFrame(book_score.items(), columns=["Book-Title", "Book-Score"]).sort_values(by="Book-Score", ascending=False)
# book_score.reset_index(drop=True).head()